In [11]:
def img2vector(filename):
    """
    将32x32的二进制图像转换为1x1024向量。
    Parameters:
        filename - 文件名
    Returns:
        returnVect - 返回的二进制图像的1x1024向量
    """
    returnVect = np.zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect
def loadImages(dirName):
    """
    加载图片
    Parameters:
        dirName - 文件夹的名字
    Returns:
        trainingMat - 数据矩阵
        hwLabels - 数据标签
    """
    from os import listdir
    hwLabels = []
    trainingFileList = listdir(r'E:\Program Files\Machine Learning\机器学习实战及配套代码\machinelearninginaction\Ch02\trainingDigits')           
    m = len(trainingFileList)
    trainingMat = np.zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]     
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9: hwLabels.append(-1)
        else: hwLabels.append(1)
        trainingMat[i,:] = img2vector(r'E:\Program Files\Machine Learning\机器学习实战及配套代码\machinelearninginaction\Ch02\trainingDigits\%s' % fileNameStr)
    return trainingMat, hwLabels 


In [12]:
import matplotlib.pyplot as plt
import numpy as np
import random

In [13]:
class optStruct:
    """
    数据结构，维护所有需要操作的值
    Parameters：
        dataMatIn - 数据矩阵
        classLabels - 数据标签
        C - 松弛变量
        toler - 容错率
        kTup - 包含核函数信息的元组,第一个参数存放核函数类别，第二个参数存放必要的核函数需要用到的参数
    """
    def __init__(self, dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn                                #数据矩阵
        self.labelMat = classLabels                        #数据标签
        self.C = C                                         #松弛变量
        self.tol = toler                                 #容错率
        self.m = np.shape(dataMatIn)[0]                 #数据矩阵行数
        self.alphas = np.mat(np.zeros((self.m,1)))         #根据矩阵行数初始化alpha参数为0   
        self.b = 0                                         #初始化b参数为0
        self.eCache = np.mat(np.zeros((self.m,2)))         #根据矩阵行数初始化虎误差缓存，第一列为是否有效的标志位，第二列为实际的误差E的值。
        self.K = np.mat(np.zeros((self.m,self.m)))        #初始化核K
        for i in range(self.m):                            #计算所有数据的核K
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)

def kernelTrans(X, A, kTup):
    """
    通过核函数将数据转换更高维的空间
    Parameters：
        X - 数据矩阵
        A - 单个数据的向量
        kTup - 包含核函数信息的元组
    Returns:
        K - 计算的核K
    """
    m,n = np.shape(X)
    K = np.mat(np.zeros((m,1)))
    if kTup[0] == 'lin': K = X * A.T                       #线性核函数,只进行内积。
    elif kTup[0] == 'rbf':                                 #高斯核函数,根据高斯核函数公式进行计算
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow*deltaRow.T
        K = np.exp(K/(-1*kTup[1]**2))                     #计算高斯核K
    else: raise NameError('核函数无法识别')
    return K                                             #返回计算的核K

In [14]:
def loadDataSet(fileName):
    """
    读取数据
    Parameters:
        fileName - 文件名
    Returns:
        dataMat - 数据矩阵
        labelMat - 数据标签
    """
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():                                     #逐行读取，滤除空格等
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])      #添加数据
        labelMat.append(float(lineArr[2]))                          #添加标签
    return dataMat,labelMat

def calcEk(oS, k):
    """
    计算误差
    Parameters：
        oS - 数据结构
        k - 标号为k的数据
    Returns:
        Ek - 标号为k的数据误差
    """
    fXk = float(np.multiply(oS.alphas,oS.labelMat).T*oS.K[:,k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek

def selectJrand(i, m):
    """
    函数说明:随机选择alpha_j的索引值

    Parameters:
        i - alpha_i的索引值
        m - alpha参数个数
    Returns:
        j - alpha_j的索引值
    """
    j = i                                 #选择一个不等于i的j
    while (j == i):
        j = int(random.uniform(0, m))
    return j

def selectJ(i, oS, Ei):
    """
    内循环启发方式2
    Parameters：
        i - 标号为i的数据的索引值
        oS - 数据结构
        Ei - 标号为i的数据误差
    Returns:
        j, maxK - 标号为j或maxK的数据的索引值
        Ej - 标号为j的数据误差
    """
    maxK = -1; maxDeltaE = 0; Ej = 0                         #初始化
    oS.eCache[i] = [1,Ei]                                      #根据Ei更新误差缓存
    validEcacheList = np.nonzero(oS.eCache[:,0].A)[0]        #返回误差不为0的数据的索引值
    if (len(validEcacheList)) > 1:                            #有不为0的误差
        for k in validEcacheList:                           #遍历,找到最大的Ek
            if k == i: continue                             #不计算i,浪费时间
            Ek = calcEk(oS, k)                                #计算Ek
            deltaE = abs(Ei - Ek)                            #计算|Ei-Ek|
            if (deltaE > maxDeltaE):                        #找到maxDeltaE
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK, Ej                                        #返回maxK,Ej
    else:                                                   #没有不为0的误差
        j = selectJrand(i, oS.m)                            #随机选择alpha_j的索引值
        Ej = calcEk(oS, j)                                    #计算Ej
    return j, Ej                                             #j,Ej

def updateEk(oS, k):
    """
    计算Ek,并更新误差缓存
    Parameters：
        oS - 数据结构
        k - 标号为k的数据的索引值
    Returns:
        无
    """
    Ek = calcEk(oS, k)                                        #计算Ek
    oS.eCache[k] = [1,Ek]                                    #更新误差缓存


def clipAlpha(aj,H,L):
    """
    修剪alpha_j
    Parameters:
        aj - alpha_j的值
        H - alpha上限
        L - alpha下限
    Returns:
        aj - 修剪后的alpah_j的值
    """
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

In [15]:
def innerL(i, oS):
    """
    优化的SMO算法
    Parameters：
        i - 标号为i的数据的索引值
        oS - 数据结构
    Returns:
        1 - 有任意一对alpha值发生变化
        0 - 没有任意一对alpha值发生变化或变化太小
    """
    #步骤1：计算误差Ei
    Ei = calcEk(oS, i)
    #优化alpha,设定一定的容错率。
    if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] > 0)):
        #使用内循环启发方式2选择alpha_j,并计算Ej
        j,Ej = selectJ(i, oS, Ei)
        #保存更新前的aplpha值，使用深拷贝
        alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
        #步骤2：计算上下界L和H
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L == H:
            print("L==H")
            return 0
        #步骤3：计算eta
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j]
        if eta >= 0:
            print("eta>=0")
            return 0
        #步骤4：更新alpha_j
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej)/eta
        #步骤5：修剪alpha_j
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        #更新Ej至误差缓存
        updateEk(oS, j)
        if (abs(oS.alphas[j] - alphaJold) < 0.00001):
            print("alpha_j变化太小")
            return 0
        #步骤6：更新alpha_i
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])
        #更新Ei至误差缓存
        updateEk(oS, i)
        #步骤7：更新b_1和b_2
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]- oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        #步骤8：根据b_1和b_2更新b
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2)/2.0
        return 1
    else:
        return 0

In [16]:
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup = ('lin',0)):
    """
    完整的线性SMO算法
    Parameters：
        dataMatIn - 数据矩阵
        classLabels - 数据标签
        C - 松弛变量
        toler - 容错率
        maxIter - 最大迭代次数
        kTup - 包含核函数信息的元组
    Returns:
        oS.b - SMO算法计算的b
        oS.alphas - SMO算法计算的alphas
    """
    oS = optStruct(np.mat(dataMatIn), np.mat(classLabels).transpose(), C, toler, kTup)                #初始化数据结构
    iter = 0                                                                                         #初始化当前迭代次数
    entireSet = True; alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):                            #遍历整个数据集都alpha也没有更新或者超过最大迭代次数,则退出循环
        alphaPairsChanged = 0
        if entireSet:                                                                                #遍历整个数据集                           
            for i in range(oS.m):       
                alphaPairsChanged += innerL(i,oS)                                                    #使用优化的SMO算法
                print("全样本遍历:第%d次迭代 样本:%d, alpha优化次数:%d" % (iter,i,alphaPairsChanged))
            iter += 1
        else:                                                                                         #遍历非边界值
            nonBoundIs = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]                        #遍历不在边界0和C的alpha
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print("非边界遍历:第%d次迭代 样本:%d, alpha优化次数:%d" % (iter,i,alphaPairsChanged))
            iter += 1
        if entireSet:                                                                                #遍历一次后改为非边界遍历
            entireSet = False
        elif (alphaPairsChanged == 0):                                                                #如果alpha没有更新,计算全样本遍历
            entireSet = True 
        print("迭代次数: %d" % iter)
    return oS.b,oS.alphas            

In [9]:
def img2vector(filename):
    """
    将32x32的二进制图像转换为1x1024向量。
    Parameters:
        filename - 文件名
    Returns:
        returnVect - 返回的二进制图像的1x1024向量
    """
    returnVect = np.zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect

In [17]:
def testDigits(kTup=('rbf', 10)):
    """
    测试函数
    Parameters:
        kTup - 包含核函数信息的元组
    Returns:
        无
    """
    dataArr,labelArr = loadImages(r'E:\Program Files\Machine Learning\机器学习实战及配套代码\machinelearninginaction\Ch02\trainingDigits')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, 10, kTup)
    datMat = np.mat(dataArr); labelMat = np.mat(labelArr).transpose()
    svInd = np.nonzero(alphas.A>0)[0]
    sVs=datMat[svInd]
    labelSV = labelMat[svInd];
    print("支持向量个数:%d" % np.shape(sVs)[0])
    m,n = np.shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * np.multiply(labelSV,alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelArr[i]): errorCount += 1
    print("训练集错误率: %.2f%%" % (float(errorCount)/m))
    dataArr,labelArr = loadImages(r'E:\Program Files\Machine Learning\机器学习实战及配套代码\machinelearninginaction\Ch02\testDigits')
    errorCount = 0
    datMat = np.mat(dataArr); labelMat = np.mat(labelArr).transpose()
    m,n = np.shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * np.multiply(labelSV,alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelArr[i]): errorCount += 1   
    print("测试集错误率: %.2f%%" % (float(errorCount)/m))

In [ ]:
testDigits(kTup=('rbf', 10))

L==H
全样本遍历:第0次迭代 样本:0, alpha优化次数:0
全样本遍历:第0次迭代 样本:1, alpha优化次数:1
全样本遍历:第0次迭代 样本:2, alpha优化次数:2
全样本遍历:第0次迭代 样本:3, alpha优化次数:3
全样本遍历:第0次迭代 样本:4, alpha优化次数:4
全样本遍历:第0次迭代 样本:5, alpha优化次数:5
L==H
全样本遍历:第0次迭代 样本:6, alpha优化次数:5
全样本遍历:第0次迭代 样本:7, alpha优化次数:6
全样本遍历:第0次迭代 样本:8, alpha优化次数:7
全样本遍历:第0次迭代 样本:9, alpha优化次数:8
全样本遍历:第0次迭代 样本:10, alpha优化次数:9
L==H
全样本遍历:第0次迭代 样本:11, alpha优化次数:9
全样本遍历:第0次迭代 样本:12, alpha优化次数:10
全样本遍历:第0次迭代 样本:13, alpha优化次数:11
全样本遍历:第0次迭代 样本:14, alpha优化次数:12
全样本遍历:第0次迭代 样本:15, alpha优化次数:13
全样本遍历:第0次迭代 样本:16, alpha优化次数:14
全样本遍历:第0次迭代 样本:17, alpha优化次数:15
全样本遍历:第0次迭代 样本:18, alpha优化次数:16
全样本遍历:第0次迭代 样本:19, alpha优化次数:16
全样本遍历:第0次迭代 样本:20, alpha优化次数:17
全样本遍历:第0次迭代 样本:21, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:22, alpha优化次数:18
全样本遍历:第0次迭代 样本:23, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:24, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:25, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:26, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:27, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:28, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:29, alpha优化次数:18
L==H
全样本遍历:第

L==H
全样本遍历:第0次迭代 样本:223, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:224, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:225, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:226, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:227, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:228, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:229, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:230, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:231, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:232, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:233, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:234, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:235, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:236, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:237, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:238, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:239, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:240, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:241, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:242, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:243, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:244, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:245, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:246, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:247, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:248, alpha优化次数:18
L==H
全样本遍历:第

L==H
全样本遍历:第0次迭代 样本:443, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:444, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:445, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:446, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:447, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:448, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:449, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:450, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:451, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:452, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:453, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:454, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:455, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:456, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:457, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:458, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:459, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:460, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:461, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:462, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:463, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:464, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:465, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:466, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:467, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:468, alpha优化次数:18
L==H
全样本遍历:第

L==H
全样本遍历:第0次迭代 样本:661, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:662, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:663, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:664, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:665, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:666, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:667, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:668, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:669, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:670, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:671, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:672, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:673, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:674, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:675, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:676, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:677, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:678, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:679, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:680, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:681, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:682, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:683, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:684, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:685, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:686, alpha优化次数:18
L==H
全样本遍历:第

L==H
全样本遍历:第0次迭代 样本:877, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:878, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:879, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:880, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:881, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:882, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:883, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:884, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:885, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:886, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:887, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:888, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:889, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:890, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:891, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:892, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:893, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:894, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:895, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:896, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:897, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:898, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:899, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:900, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:901, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:902, alpha优化次数:18
L==H
全样本遍历:第

...

alpha_j变化太小
全样本遍历:第3次迭代 样本:1024, alpha优化次数:0
全样本遍历:第3次迭代 样本:1025, alpha优化次数:0
全样本遍历:第3次迭代 样本:1026, alpha优化次数:0
L==H
全样本遍历:第3次迭代 样本:1027, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1028, alpha优化次数:0
全样本遍历:第3次迭代 样本:1029, alpha优化次数:0
全样本遍历:第3次迭代 样本:1030, alpha优化次数:0
全样本遍历:第3次迭代 样本:1031, alpha优化次数:0
全样本遍历:第3次迭代 样本:1032, alpha优化次数:0
全样本遍历:第3次迭代 样本:1033, alpha优化次数:0
全样本遍历:第3次迭代 样本:1034, alpha优化次数:0
全样本遍历:第3次迭代 样本:1035, alpha优化次数:0
全样本遍历:第3次迭代 样本:1036, alpha优化次数:0
全样本遍历:第3次迭代 样本:1037, alpha优化次数:0
全样本遍历:第3次迭代 样本:1038, alpha优化次数:0
全样本遍历:第3次迭代 样本:1039, alpha优化次数:0
全样本遍历:第3次迭代 样本:1040, alpha优化次数:0
L==H
全样本遍历:第3次迭代 样本:1041, alpha优化次数:0
L==H
全样本遍历:第3次迭代 样本:1042, alpha优化次数:0
L==H
全样本遍历:第3次迭代 样本:1043, alpha优化次数:0
全样本遍历:第3次迭代 样本:1044, alpha优化次数:0
L==H
全样本遍历:第3次迭代 样本:1045, alpha优化次数:0
L==H
全样本遍历:第3次迭代 样本:1046, alpha优化次数:0
全样本遍历:第3次迭代 样本:1047, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1048, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1049, alpha优化次数:0
全样本遍历:第3次迭代 样本:1050, alpha优化次数:0
全样本遍历:第3次迭代 样本:1051, alpha优化次数:

全样本遍历:第3次迭代 样本:1261, alpha优化次数:0
全样本遍历:第3次迭代 样本:1262, alpha优化次数:0
全样本遍历:第3次迭代 样本:1263, alpha优化次数:0
全样本遍历:第3次迭代 样本:1264, alpha优化次数:0
全样本遍历:第3次迭代 样本:1265, alpha优化次数:0
全样本遍历:第3次迭代 样本:1266, alpha优化次数:0
全样本遍历:第3次迭代 样本:1267, alpha优化次数:0
全样本遍历:第3次迭代 样本:1268, alpha优化次数:0
全样本遍历:第3次迭代 样本:1269, alpha优化次数:0
全样本遍历:第3次迭代 样本:1270, alpha优化次数:0
全样本遍历:第3次迭代 样本:1271, alpha优化次数:0
全样本遍历:第3次迭代 样本:1272, alpha优化次数:0
全样本遍历:第3次迭代 样本:1273, alpha优化次数:0
全样本遍历:第3次迭代 样本:1274, alpha优化次数:0
全样本遍历:第3次迭代 样本:1275, alpha优化次数:0
全样本遍历:第3次迭代 样本:1276, alpha优化次数:0
全样本遍历:第3次迭代 样本:1277, alpha优化次数:0
全样本遍历:第3次迭代 样本:1278, alpha优化次数:0
全样本遍历:第3次迭代 样本:1279, alpha优化次数:0
全样本遍历:第3次迭代 样本:1280, alpha优化次数:0
全样本遍历:第3次迭代 样本:1281, alpha优化次数:0
全样本遍历:第3次迭代 样本:1282, alpha优化次数:0
全样本遍历:第3次迭代 样本:1283, alpha优化次数:0
全样本遍历:第3次迭代 样本:1284, alpha优化次数:0
全样本遍历:第3次迭代 样本:1285, alpha优化次数:0
全样本遍历:第3次迭代 样本:1286, alpha优化次数:0
全样本遍历:第3次迭代 样本:1287, alpha优化次数:0
全样本遍历:第3次迭代 样本:1288, alpha优化次数:0
全样本遍历:第3次迭代 样本:1289, alpha优化次数:0
全样本遍历:第3次迭代 样本:1290, alpha优化次数:0
全样本遍历:第3次迭

L==H
全样本遍历:第3次迭代 样本:1503, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1504, alpha优化次数:0
全样本遍历:第3次迭代 样本:1505, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1506, alpha优化次数:0
全样本遍历:第3次迭代 样本:1507, alpha优化次数:0
全样本遍历:第3次迭代 样本:1508, alpha优化次数:0
全样本遍历:第3次迭代 样本:1509, alpha优化次数:0
全样本遍历:第3次迭代 样本:1510, alpha优化次数:0
全样本遍历:第3次迭代 样本:1511, alpha优化次数:0
全样本遍历:第3次迭代 样本:1512, alpha优化次数:0
全样本遍历:第3次迭代 样本:1513, alpha优化次数:0
全样本遍历:第3次迭代 样本:1514, alpha优化次数:0
全样本遍历:第3次迭代 样本:1515, alpha优化次数:0
全样本遍历:第3次迭代 样本:1516, alpha优化次数:0
全样本遍历:第3次迭代 样本:1517, alpha优化次数:0
全样本遍历:第3次迭代 样本:1518, alpha优化次数:0
全样本遍历:第3次迭代 样本:1519, alpha优化次数:0
全样本遍历:第3次迭代 样本:1520, alpha优化次数:0
全样本遍历:第3次迭代 样本:1521, alpha优化次数:0
全样本遍历:第3次迭代 样本:1522, alpha优化次数:0
全样本遍历:第3次迭代 样本:1523, alpha优化次数:0
全样本遍历:第3次迭代 样本:1524, alpha优化次数:0
全样本遍历:第3次迭代 样本:1525, alpha优化次数:0
全样本遍历:第3次迭代 样本:1526, alpha优化次数:0
全样本遍历:第3次迭代 样本:1527, alpha优化次数:0
L==H
全样本遍历:第3次迭代 样本:1528, alpha优化次数:0
全样本遍历:第3次迭代 样本:1529, alpha优化次数:0
全样本遍历:第3次迭代 样本:1530, alpha优化次数:0
全样本遍历:第3次迭代 样本:1531, alpha优化次数:0
全样本遍历:第3次

alpha_j变化太小
全样本遍历:第3次迭代 样本:1733, alpha优化次数:0
全样本遍历:第3次迭代 样本:1734, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1735, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1736, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1737, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1738, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1739, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1740, alpha优化次数:0
全样本遍历:第3次迭代 样本:1741, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1742, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1743, alpha优化次数:0
全样本遍历:第3次迭代 样本:1744, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1745, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1746, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1747, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1748, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1749, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1750, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1751, alpha优化次数:0
全样本遍历:第3次迭代 样本:1752, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1753, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1754, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1755, alpha优化次数:0
alpha_j变化太小
全

alpha_j变化太小
全样本遍历:第3次迭代 样本:1923, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1924, alpha优化次数:0
全样本遍历:第3次迭代 样本:1925, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1926, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1927, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1928, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1929, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1930, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1931, alpha优化次数:0
全样本遍历:第3次迭代 样本:1932, alpha优化次数:0
alpha_j变化太小
全样本遍历:第3次迭代 样本:1933, alpha优化次数:0
迭代次数: 4
支持向量个数:298
训练集错误率: 0.00%
测试集错误率: 0.00%


In [ ]:
testDigits(kTup=('rbf', 10))

L==H
全样本遍历:第0次迭代 样本:0, alpha优化次数:0
全样本遍历:第0次迭代 样本:1, alpha优化次数:1
全样本遍历:第0次迭代 样本:2, alpha优化次数:2
全样本遍历:第0次迭代 样本:3, alpha优化次数:3
全样本遍历:第0次迭代 样本:4, alpha优化次数:4
全样本遍历:第0次迭代 样本:5, alpha优化次数:5
L==H
全样本遍历:第0次迭代 样本:6, alpha优化次数:5
全样本遍历:第0次迭代 样本:7, alpha优化次数:6
全样本遍历:第0次迭代 样本:8, alpha优化次数:7
全样本遍历:第0次迭代 样本:9, alpha优化次数:8
全样本遍历:第0次迭代 样本:10, alpha优化次数:9
L==H
全样本遍历:第0次迭代 样本:11, alpha优化次数:9
全样本遍历:第0次迭代 样本:12, alpha优化次数:10
全样本遍历:第0次迭代 样本:13, alpha优化次数:11
全样本遍历:第0次迭代 样本:14, alpha优化次数:12
全样本遍历:第0次迭代 样本:15, alpha优化次数:13
全样本遍历:第0次迭代 样本:16, alpha优化次数:14
全样本遍历:第0次迭代 样本:17, alpha优化次数:15
全样本遍历:第0次迭代 样本:18, alpha优化次数:16
全样本遍历:第0次迭代 样本:19, alpha优化次数:16
全样本遍历:第0次迭代 样本:20, alpha优化次数:17
全样本遍历:第0次迭代 样本:21, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:22, alpha优化次数:18
全样本遍历:第0次迭代 样本:23, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:24, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:25, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:26, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:27, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:28, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:29, alpha优化次数:18
L==H
全样本遍历:第

L==H
全样本遍历:第0次迭代 样本:231, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:232, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:233, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:234, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:235, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:236, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:237, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:238, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:239, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:240, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:241, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:242, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:243, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:244, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:245, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:246, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:247, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:248, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:249, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:250, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:251, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:252, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:253, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:254, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:255, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:256, alpha优化次数:18
L==H
全样本遍历:第

L==H
全样本遍历:第0次迭代 样本:450, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:451, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:452, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:453, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:454, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:455, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:456, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:457, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:458, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:459, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:460, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:461, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:462, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:463, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:464, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:465, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:466, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:467, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:468, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:469, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:470, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:471, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:472, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:473, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:474, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:475, alpha优化次数:18
L==H
全样本遍历:第

L==H
全样本遍历:第0次迭代 样本:669, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:670, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:671, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:672, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:673, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:674, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:675, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:676, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:677, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:678, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:679, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:680, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:681, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:682, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:683, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:684, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:685, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:686, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:687, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:688, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:689, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:690, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:691, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:692, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:693, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:694, alpha优化次数:18
L==H
全样本遍历:第

L==H
全样本遍历:第0次迭代 样本:885, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:886, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:887, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:888, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:889, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:890, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:891, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:892, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:893, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:894, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:895, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:896, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:897, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:898, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:899, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:900, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:901, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:902, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:903, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:904, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:905, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:906, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:907, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:908, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:909, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:910, alpha优化次数:18
L==H
全样本遍历:第

L==H
全样本遍历:第0次迭代 样本:1099, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1100, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1101, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1102, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1103, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1104, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1105, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1106, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1107, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1108, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1109, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1110, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1111, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1112, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1113, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1114, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1115, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1116, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1117, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1118, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1119, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1120, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1121, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1122, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1123, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1124,

L==H
全样本遍历:第0次迭代 样本:1310, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1311, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1312, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1313, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1314, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1315, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1316, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1317, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1318, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1319, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1320, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1321, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1322, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1323, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1324, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1325, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1326, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1327, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1328, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1329, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1330, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1331, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1332, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1333, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1334, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1335,

L==H
全样本遍历:第0次迭代 样本:1522, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1523, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1524, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1525, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1526, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1527, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1528, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1529, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1530, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1531, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1532, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1533, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1534, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1535, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1536, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1537, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1538, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1539, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1540, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1541, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1542, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1543, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1544, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1545, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1546, alpha优化次数:18
L==H
全样本遍历:第0次迭代 样本:1547,

全样本遍历:第0次迭代 样本:1734, alpha优化次数:23
全样本遍历:第0次迭代 样本:1735, alpha优化次数:24
全样本遍历:第0次迭代 样本:1736, alpha优化次数:25
全样本遍历:第0次迭代 样本:1737, alpha优化次数:26
全样本遍历:第0次迭代 样本:1738, alpha优化次数:27
全样本遍历:第0次迭代 样本:1739, alpha优化次数:28
全样本遍历:第0次迭代 样本:1740, alpha优化次数:29
全样本遍历:第0次迭代 样本:1741, alpha优化次数:30
全样本遍历:第0次迭代 样本:1742, alpha优化次数:31
全样本遍历:第0次迭代 样本:1743, alpha优化次数:32
全样本遍历:第0次迭代 样本:1744, alpha优化次数:33
全样本遍历:第0次迭代 样本:1745, alpha优化次数:34
全样本遍历:第0次迭代 样本:1746, alpha优化次数:35
全样本遍历:第0次迭代 样本:1747, alpha优化次数:36
全样本遍历:第0次迭代 样本:1748, alpha优化次数:37
全样本遍历:第0次迭代 样本:1749, alpha优化次数:38
全样本遍历:第0次迭代 样本:1750, alpha优化次数:39
全样本遍历:第0次迭代 样本:1751, alpha优化次数:40
全样本遍历:第0次迭代 样本:1752, alpha优化次数:41
全样本遍历:第0次迭代 样本:1753, alpha优化次数:42
全样本遍历:第0次迭代 样本:1754, alpha优化次数:43
全样本遍历:第0次迭代 样本:1755, alpha优化次数:44
alpha_j变化太小
全样本遍历:第0次迭代 样本:1756, alpha优化次数:44
全样本遍历:第0次迭代 样本:1757, alpha优化次数:45
全样本遍历:第0次迭代 样本:1758, alpha优化次数:46
全样本遍历:第0次迭代 样本:1759, alpha优化次数:47
全样本遍历:第0次迭代 样本:1760, alpha优化次数:48
全样本遍历:第0次迭代 样本:1761, alpha优化次数:49
全样本遍历:第0次迭代 样本:1762, alpha优化次数:50
全样

alpha_j变化太小
非边界遍历:第1次迭代 样本:318, alpha优化次数:18
alpha_j变化太小
非边界遍历:第1次迭代 样本:319, alpha优化次数:18
alpha_j变化太小
非边界遍历:第1次迭代 样本:424, alpha优化次数:18
alpha_j变化太小
非边界遍历:第1次迭代 样本:485, alpha优化次数:18
alpha_j变化太小
非边界遍历:第1次迭代 样本:490, alpha优化次数:18
alpha_j变化太小
非边界遍历:第1次迭代 样本:551, alpha优化次数:18
非边界遍历:第1次迭代 样本:593, alpha优化次数:19
alpha_j变化太小
非边界遍历:第1次迭代 样本:601, alpha优化次数:19
alpha_j变化太小
非边界遍历:第1次迭代 样本:609, alpha优化次数:19
alpha_j变化太小
非边界遍历:第1次迭代 样本:610, alpha优化次数:19
alpha_j变化太小
非边界遍历:第1次迭代 样本:612, alpha优化次数:19
非边界遍历:第1次迭代 样本:618, alpha优化次数:20
非边界遍历:第1次迭代 样本:625, alpha优化次数:21
alpha_j变化太小
非边界遍历:第1次迭代 样本:654, alpha优化次数:21
alpha_j变化太小
非边界遍历:第1次迭代 样本:655, alpha优化次数:21
alpha_j变化太小
非边界遍历:第1次迭代 样本:660, alpha优化次数:21
非边界遍历:第1次迭代 样本:682, alpha优化次数:22
alpha_j变化太小
非边界遍历:第1次迭代 样本:691, alpha优化次数:22
alpha_j变化太小
非边界遍历:第1次迭代 样本:707, alpha优化次数:22
alpha_j变化太小
非边界遍历:第1次迭代 样本:724, alpha优化次数:22
alpha_j变化太小
非边界遍历:第1次迭代 样本:727, alpha优化次数:22
alpha_j变化太小
非边界遍历:第1次迭代 样本:744, alpha优化次数:22
非边界遍历:第1次迭代 样本:746, alpha优化次数:23
alpha_j变化太小
非边界遍历:第1次迭代 样